In [ ]:
import google.generativeai as genai
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Markdown
import json
import random
from datetime import datetime

# ==========================================
# 1. CONFIGURATION & SETUP
# ==========================================

import os
API_KEY = os.environ.get("GEMINI_API_KEY")

# Configure the AI
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# File to store student progress
LOG_FILE = 'collocation_log.csv'

# Initialize the log file if it doesn't exist
try:
    log_df = pd.read_csv(LOG_FILE)
except FileNotFoundError:
    log_df = pd.DataFrame(columns=['date', 'theme', 'collocation', 'example', 'status'])
    log_df.to_csv(LOG_FILE, index=False)

# ==========================================
# 2. AI & DATA FUNCTIONS (The "Brain")
# ==========================================

def get_gemini_json(prompt):
    """Helper to get clean JSON from Gemini"""
    try:
        response = model.generate_content(
            prompt,
            generation_config={"response_mime_type": "application/json"}
        )
        return json.loads(response.text)
    except Exception as e:
        return {"error": str(e)}

def generate_new_set(theme=None):
    """Generates 5 new collocations based on a theme"""
    if not theme:
        themes = ["Business", "Travel", "Emotions", "Time", "Money", "Technology", "Friendship"]
        theme = random.choice(themes)
    
    prompt = f"""
    Generate 5 high-frequency English collocations related to the theme: '{theme}'.
    Focus on B2/C1 level phrases (Verb+Noun or Adj+Noun).
    
    Output strictly this JSON structure:
    {{
        "theme": "{theme}",
        "collocations": [
            {{"pair": "collocation 1", "definition": "short definition", "example": "sentence example"}},
            {{"pair": "collocation 2", "definition": "short definition", "example": "sentence example"}}
        ]
    }}
    """
    return get_gemini_json(prompt)

def check_writing(text, required_words):
    """Checks if the student used the collocations correctly"""
    prompt = f"""
    A student was asked to write a paragraph using these specific collocations: {required_words}.
    Student text: "{text}"
    
    Analyze the text. Did they use the collocations? Are they used naturally?
    Output strictly this JSON structure:
    {{
        "feedback": "General feedback on their writing style.",
        "corrections": [
            {{"phrase": "phrase used", "status": "Correct" or "Incorrect", "comment": "Explanation"}}
        ],
        "score": "Give a score out of 10"
    }}
    """
    return get_gemini_json(prompt)

def save_to_log(data):
    """Saves new collocations to the CSV file"""
    global log_df
    new_rows = []
    for item in data['collocations']:
        new_rows.append({
            'date': datetime.now().strftime("%Y-%m-%d %H:%M"),
            'theme': data['theme'],
            'collocation': item['pair'],
            'example': item['example'],
            'status': 'new'
        })
    
    new_df = pd.DataFrame(new_rows)
    log_df = pd.concat([log_df, new_df], ignore_index=True)
    log_df.to_csv(LOG_FILE, index=False)
    return new_rows

# ==========================================
# 3. UI LAYOUT & INTERACTION (The "Face")
# ==========================================

# -- Output Area (Where content appears) --
content_out = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '20px', 'min_height': '400px'})

# -- Header --
header = widgets.HTML("<h1>🎓 The Collocation Coach</h1><p>Master English word pairings naturally.</p>")

# -- Button Styling --
btn_layout = widgets.Layout(width='200px', height='50px', margin='5px')
btn_style = {'font_weight': 'bold', 'font_size': '14px'}

# -- VIEW: New Collocations --
def view_new(b):
    with content_out:
        clear_output()
        display(widgets.HTML("<h3>⏳ Generating fresh collocations... please wait...</h3>"))
        
        data = generate_new_set()
        if "error" in data:
            display(widgets.HTML(f"<b style='color:red'>Error: {data['error']}</b>"))
            return

        # Save to DB
        save_to_log(data)
        
        # Display Result
        html = f"<h2>✨ Theme: {data['theme']}</h2><ul>"
        for item in data['collocations']:
            html += f"<li style='margin-bottom:10px; font-size:16px'><b>{item['pair']}</b>: {item['definition']}<br><i style='color:#666'>Ex: {item['example']}</i></li>"
        html += "</ul><br><b>✅ Saved to your vocabulary log!</b>"
        
        display(widgets.HTML(html))

# -- VIEW: Review Mode --
def view_review(b):
    with content_out:
        clear_output()
        if log_df.empty:
            display(widgets.HTML("<b>⚠️ No collocations saved yet. Go learn some 'New' ones first!</b>"))
            return
            
        # Get last 5 items
        review_set = log_df.tail(5).to_dict('records')
        
        html = "<h2>⏪ Review: Your Last 5 Collocations</h2><p>Can you remember the sentences?</p><ul>"
        for item in review_set:
            html += f"<li style='margin-bottom:15px; font-size:16px'><b>{item['collocation']}</b> <span style='font-size:12px; background:#eee; padding:2px'>({item['theme']})</span></li>"
        html += "</ul>"
        display(widgets.HTML(html))

# -- VIEW: Production (Writing Task) --
def view_produce(b):
    with content_out:
        clear_output()
        if log_df.empty:
            display(widgets.HTML("<b>⚠️ Go learn some collocations first!</b>"))
            return

        # Pick random 3 words from history
        words = log_df['collocation'].sample(n=min(3, len(log_df))).tolist()
        word_str = ", ".join([f"<b>'{w}'</b>" for w in words])
        
        display(widgets.HTML(f"<h3>✍️ Writing Challenge</h3><p>Write a short paragraph using these 3 phrases: {word_str}</p>"))
        
        text_area = widgets.Textarea(placeholder="Type your paragraph here...", layout=widgets.Layout(width='100%', height='150px'))
        submit_btn = widgets.Button(description="Check My Writing", button_style='success')
        
        def on_submit_writing(c):
            submit_btn.disabled = True
            submit_btn.description = "Checking..."
            result = check_writing(text_area.value, words)
            
            # Show Feedback
            res_html = f"<h3>📝 Feedback (Score: {result['score']})</h3><p>{result['feedback']}</p><ul>"
            for correction in result['corrections']:
                color = "green" if correction['status'] == "Correct" else "red"
                res_html += f"<li style='color:{color}'><b>{correction['phrase']}:</b> {correction['status']} - {correction['comment']}</li>"
            res_html += "</ul>"
            
            display(widgets.HTML(res_html))
            submit_btn.disabled = False
            submit_btn.description = "Check My Writing"

        submit_btn.on_click(on_submit_writing)
        display(text_area, submit_btn)

# -- VIEW: Quiz Mode --
def view_quiz(b):
    with content_out:
        clear_output()
        if len(log_df) < 4:
            display(widgets.HTML("<b>⚠️ You need at least 4 saved collocations to take a quiz.</b>"))
            return
            
        # Create a simple question
        target = log_df.sample(1).iloc[0]
        correct = target['collocation']
        
        # Split word to make a gap (simple heuristic: remove last word)
        parts = correct.split()
        if len(parts) > 1:
            gap_word = parts[-1]
            base_phrase = " ".join(parts[:-1]) + " _______"
        else:
            gap_word = correct
            base_phrase = "_______"
            
        display(widgets.HTML(f"<h3>❓ Quiz Time</h3><p>Complete the collocation:</p><h2>{base_phrase}</h2>"))
        
        options = [gap_word] + ["make", "do", "take", "get", "high", "big"] # Simple distractors
        options = list(set(options)) # unique
        random.shuffle(options)
        
        opts = widgets.RadioButtons(options=options[:4], description='Choose:', disabled=False)
        check_btn = widgets.Button(description="Submit Answer", button_style='info')
        
        def check_answer(c):
            if opts.value == gap_word:
                display(widgets.HTML("<h3 style='color:green'>✅ Correct!</h3>"))
            else:
                display(widgets.HTML(f"<h3 style='color:red'>❌ Incorrect. The correct phrase is: {correct}</h3>"))
                
        check_btn.on_click(check_answer)
        display(opts, check_btn)

# -- BUTTON CREATION --
btn_new = widgets.Button(description="📖 New Collocations", layout=btn_layout, button_style='primary')
btn_review = widgets.Button(description="⏪ Review Last 5", layout=btn_layout, button_style='info')
btn_produce = widgets.Button(description="✍️ Practice Writing", layout=btn_layout, button_style='warning')
btn_quiz = widgets.Button(description="📝 Quick Quiz", layout=btn_layout, button_style='success')

# Link buttons to functions
btn_new.on_click(view_new)
btn_review.on_click(view_review)
btn_produce.on_click(view_produce)
btn_quiz.on_click(view_quiz)

# -- FINAL DASHBOARD ASSEMBLY --
menu = widgets.HBox([btn_new, btn_review, btn_produce, btn_quiz], layout=widgets.Layout(justify_content='center'))
dashboard = widgets.VBox([header, menu, content_out])

# Show the app
display(dashboard)